In [24]:
import nltk
import numpy as np
import pandas as pd
from collections import OrderedDict
import string, parse,json
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC

In [2]:
from allennlp.modules.elmo import Elmo, batch_to_ids
options_file = "./elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "./elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0) #1 representation only

In [25]:
removables = []
removables += nltk.corpus.stopwords.words('english')
removables += [w.lower() for w in nltk.corpus.names.words()]
removables += ['hi','trump']

lemmatizer = nltk.stem.WordNetLemmatizer()
#data fed to vectorizer is already cleaned but maybe TODO use their params instead
#vectorizer = TfidfVectorizer(max_features=400)
vectorizer = CountVectorizer(max_features=400)

In [4]:
#much info was lost
#after more reading, i realised i never needed to do this, will just use tfidfvectorizer
def filterSent(s,freq=None):
    s = s.lower()
    s = nltk.word_tokenize(s)
    s = tuple(lemmatizer.lemmatize(w) for w in s if w.isalpha())
    s = tuple(w for w in s if w not in removables)
    if freq != None:
        for w in s: 
            if w in freq: freq[w] += 1
            else: freq[w] = 1
    return ' '.join(s)
filterSent("hi there my name is jOhn henry battery car dead better programming")

'name battery car dead better programming'

In [5]:
#freq = {}
formatter = lambda x:(filterSent(x.statement),liar_score[x.label])
#freq = OrderedDict(sorted(freq.items(),key=lambda x:x[1],reverse=True))
#kept = list(freq.keys())[:800]

In [6]:
path_to_raw = './raw/'

In [26]:
liar_headers = ('id','label','statement','subjects','speaker','job_title','state','party','barely_true_counts','false_counts','half_true_counts','mostly_true_counts','pants_on_fire_counts','context')
liar_train = pd.read_csv(path_to_raw+'liar_dataset/train.tsv',delimiter='\t',header=None,names=liar_headers)
#counts refer to the speaker's reliability as a whole, interestingly excluding true counts. 
#will only be using statement & label
#print(set(liar_train.label))
#assuming linear scale
liar_score = {'true':1,
              'mostly-true':1,
              'half-true':1,
              'barely-true':0,
              'false':0,
              'pants-fire':0
             }
traindf = liar_train.loc[:,('statement','label')].apply(formatter,axis=1,result_type='broadcast')
ndata = vectorizer.fit_transform(traindf.statement)

In [20]:
liar_test = pd.read_csv(path_to_raw+'liar_dataset/test.tsv',delimiter='\t',header=None,names=liar_headers)
liar_score = {'true':1,
              'mostly-true':2,
              'half-true':3,
              'barely-true':4,
              'false':5,
              'pants-fire':6
             }
testdf = liar_test.loc[:,('statement','label')].apply(formatter,axis=1,result_type='broadcast')
tdata = vectorizer.transform(testdf.statement)

In [27]:
clf = SVC(kernel='linear')
clf.fit(ndata.toarray(),np.array(traindf.label,dtype=int))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [13]:
#I would try this if i hadnt almost immediately crashed from lag. maybe its cause its padded. ill take data points within a length range then
sentences = traindf.statement.str.split()
character_ids = batch_to_ids(sentences)
embeddings = elmo(character_ids) #[representation][sent_no][word_no (padded)]

RuntimeError: $ Torch: not enough memory: you tried to allocate 7GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [18]:
sentences[20 < sentences.map(len) < 50]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [28]:
cor = 0
for pred,tgt in zip(clf.predict(tdata.toarray()),np.array(testdf.label)):
    if pred==tgt: cor += 1
cor/len(testdf)

0.13654301499605367

In [31]:
vectorizer.get_feature_names()

['abortion',
 'according',
 'act',
 'actually',
 'administration',
 'affordable',
 'agency',
 'ago',
 'allow',
 'allowed',
 'almost',
 'already',
 'also',
 'amendment',
 'america',
 'american',
 'among',
 'amount',
 'attack',
 'average',
 'away',
 'back',
 'background',
 'ban',
 'bank',
 'barack',
 'benefit',
 'big',
 'billion',
 'black',
 'board',
 'border',
 'break',
 'brown',
 'budget',
 'bush',
 'business',
 'buy',
 'california',
 'called',
 'came',
 'campaign',
 'candidate',
 'cant',
 'car',
 'care',
 'cent',
 'change',
 'check',
 'child',
 'china',
 'citizen',
 'city',
 'class',
 'college',
 'combined',
 'come',
 'community',
 'company',
 'congress',
 'congressional',
 'congressman',
 'control',
 'cost',
 'could',
 'country',
 'county',
 'court',
 'coverage',
 'create',
 'created',
 'credit',
 'crime',
 'criminal',
 'current',
 'cut',
 'day',
 'deal',
 'death',
 'debate',
 'debt',
 'decade',
 'defense',
 'deficit',
 'democrat',
 'democratic',
 'department',
 'didnt',
 'district',

In [232]:
fmt_string = '{:f}={}'
ndata = []
with open('test.txt','w') as f:
    for _,row in df.iterrows():
        tup = tuple(w for w in row.statement if w in kept)
        if len(tup) < 10: continue
        print(fmt_string.format(row.label,json.dumps(tup)),file=f)
        ndata.append((tup,row.label))

In [233]:
fdata = {
    'w2idx':dict((k,v) for v,k in enumerate(kept)),
    'data':ndata
}

In [234]:
with open('test.json','w') as f:
    json.dump(fdata,f)

In [235]:
maxlen = 0
largest = None
for tup,t in fdata['data']:
    if len(tup) > maxlen:
        maxlen = len(tup)
        largest = tup

In [236]:
largest

('hospital',
 'doctor',
 'used',
 'far',
 'country',
 'many',
 'country',
 'former',
 'governor',
 'massachusetts',
 'republican',
 'news',
 'obamacare',
 'cut',
 'senior',
 'medicare',
 'medicare',
 'republican',
 'republican',
 'campaign',
 'email',
 'many',
 'federal',
 'employee',
 'cost',
 'taxpayer',
 'million',
 'spends',
 'million',
 'year',
 'child',
 'education',
 'democrat',
 'campaign',
 'milwaukee',
 'county',
 'citizen',
 'point',
 'center',
 'mass',
 'best',
 'option',
 'crime',
 'gun',
 'wisconsin',
 'campaign',
 'ad',
 'almost',
 'percent',
 'total',
 'income',
 'planned',
 'parenthood',
 'abortion',
 'abortion',
 'state',
 'representative',
 'republican',
 'committee',
 'united',
 'state',
 'ha',
 'highest',
 'rate',
 'poverty',
 'almost',
 'country',
 'earth',
 'child',
 'poverty',
 'senator',
 'independent',
 'democratic',
 'debate',
 'governor',
 'program',
 'allowed',
 'undocumented',
 'people',
 'use',
 'personal',
 'tax',
 'number',
 'receive',
 'driver',
 'lice